In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train = pd.read_csv('./data/train.csv')

In [3]:
test = pd.read_csv('./data/test_a.csv')

In [4]:
target = train.label

In [5]:
data = pd.concat([train.drop('label',1), test],0)

In [6]:
nancol = data.isnull().sum().sort_values(ascending=False)

In [7]:
data.shape

(1486399, 299)

In [8]:
dropcol = (nancol/data.shape[0])>0.1

In [9]:
dropcoldata = data.drop(dropcol[dropcol==True].index,1)

In [10]:
finaldata = dropcoldata.drop('id',1)

In [11]:
finaldata.head(5)

,date,f1,f2,f3,f4,f6,f7,f8,f9,f10,...,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253
0,20171103,0,0,0,0,0,5,1,1,1,...,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0
1,20170917,0,1,1,1,1,5,2,1,1,...,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0
2,20171022,0,0,1,0,0,6,1,0,0,...,2.0,2.0,2.0,7.0,13.0,2.0,1.0,2.0,4.0,6.0
3,20171029,0,0,0,1,1,4,1,2,2,...,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0
4,20171002,1,1,0,1,1,5,1,2,0,...,2.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0


In [12]:
train.shape[0]

994731

In [18]:
import xgboost as xgb
import lightgbm as lg

In [14]:
X_train = finaldata[:900000]
Y_train = target[:900000]
X_val = finaldata[900000:994731]
Y_val =target[900000:994731]
X_test = finaldata[994731:]

In [19]:
cl = xgb.XGBClassifier()
lg = lg.LGBMClassifier()

In [16]:
cl.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
lg.fit(X_train,Y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [17]:
cl.score(X_val,Y_val)

/home/ryan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9839967909132175

In [66]:
Y_test = cl.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [70]:
train.head()

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,...,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,...,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
2,1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...,0,20171022,0,0,1,0,100102.0,0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9fa009724ee7ff9d688ae321304fbc78f608cdabbfdd2b...,0,20171029,0,0,0,1,100807.0,1,4,...,302.0,322.0,341.0,57.0,251.0,175.0,176.0,176.0,49.0,150.0
4,1da482485d7e8bcefae7e9d0d1167cec3ac111cfa71d8b...,0,20171002,1,1,0,1,100805.0,1,5,...,302.0,301.0,301.0,74.0,302.0,182.0,181.0,182.0,51.0,181.0


In [72]:
submission = pd.DataFrame({
        "id": test.id,
        "score": Y_test
    })
submission.to_csv('submit.csv', index=False)